In [24]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [25]:
data = pd.read_csv("data/train.csv")
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [27]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    return np.exp(Z) / sum(np.exp(Z))

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def deriv_ReLU(Z):
    return Z > 0

def back_prop(Z1, A1, Z2, A2, W2, X, Y):
    one_hot_y = one_hot(Y)
    dZ2 = A2 - one_hot_y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2 

In [28]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 50 == 0:
            print(f"Iteration: {i}, Accuracy: {get_accuracy(get_predictions(A2), Y)}")
    return W1, b1, W2, b2

In [29]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 500, 0.1)

[7 7 2 ... 0 0 5] [0 2 5 ... 0 2 0]
Iteration: 0, Accuracy: 0.09034146341463414
[0 9 2 ... 0 2 0] [0 2 5 ... 0 2 0]
Iteration: 50, Accuracy: 0.40614634146341466
[0 4 2 ... 0 2 0] [0 2 5 ... 0 2 0]
Iteration: 100, Accuracy: 0.5783170731707317
[0 6 2 ... 0 2 0] [0 2 5 ... 0 2 0]
Iteration: 150, Accuracy: 0.6804878048780488
[0 6 2 ... 0 2 0] [0 2 5 ... 0 2 0]
Iteration: 200, Accuracy: 0.741390243902439
[0 6 6 ... 0 2 0] [0 2 5 ... 0 2 0]
Iteration: 250, Accuracy: 0.7733658536585366
[0 6 6 ... 0 2 0] [0 2 5 ... 0 2 0]
Iteration: 300, Accuracy: 0.7924390243902439
[0 6 6 ... 0 2 0] [0 2 5 ... 0 2 0]
Iteration: 350, Accuracy: 0.807219512195122
[0 6 5 ... 0 2 0] [0 2 5 ... 0 2 0]
Iteration: 400, Accuracy: 0.8183414634146341
[0 6 5 ... 0 2 0] [0 2 5 ... 0 2 0]
Iteration: 450, Accuracy: 0.8278292682926829


In [30]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print(f"Predication: {prediction}, Label: {label}")

    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [37]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)

[4 2 1 6 5 5 6 8 0 7 0 7 3 8 2 9 6 7 3 4 3 2 4 5 5 5 4 8 1 9 7 4 5 6 2 9 3
 4 8 2 1 0 4 3 5 6 3 3 1 7 8 7 2 9 7 8 6 9 7 0 7 9 0 7 4 4 4 8 6 9 8 6 8 7
 4 3 5 1 9 4 3 4 4 0 8 7 5 3 8 2 0 2 6 2 0 6 6 0 1 9 8 3 3 4 1 8 7 5 3 3 6
 7 0 1 7 3 2 7 7 1 4 7 2 2 0 1 3 3 0 6 3 3 4 3 8 6 4 9 4 8 8 0 3 9 7 3 4 4
 7 3 8 6 8 1 6 7 3 3 4 9 5 5 2 0 8 6 2 9 7 4 2 6 6 4 9 0 9 8 5 0 0 3 5 9 6
 4 1 2 2 5 9 7 1 8 5 5 6 8 0 3 6 1 9 0 7 5 9 9 8 7 9 1 9 1 9 0 8 8 3 2 9 9
 7 9 9 3 1 3 0 6 6 0 1 7 7 0 4 4 7 9 8 8 6 2 6 8 7 0 6 4 7 3 0 6 9 2 4 3 6
 1 4 7 0 6 2 9 5 1 4 4 6 7 0 2 7 4 1 4 2 2 2 5 8 7 5 2 1 2 8 6 1 7 2 9 7 2
 6 0 8 9 3 5 6 1 4 4 0 2 3 2 2 0 0 0 6 5 4 1 8 9 9 0 5 4 6 1 9 9 2 8 7 4 0
 0 7 7 4 0 1 9 4 4 2 5 1 8 1 3 4 6 4 4 7 6 9 3 5 8 3 2 7 1 7 7 2 6 5 3 5 7
 5 5 7 1 2 4 8 3 4 7 6 5 3 1 0 6 8 2 5 4 9 5 3 4 9 5 0 0 5 9 7 3 1 0 7 9 8
 3 5 6 8 8 8 8 2 1 0 0 9 7 6 8 3 7 4 3 2 2 2 2 5 3 3 2 1 2 7 5 8 0 1 1 2 9
 8 8 1 0 5 6 6 8 2 0 6 3 4 3 7 6 9 8 1 9 7 0 8 2 3 4 2 3 7 6 7 7 7 0 0 8 0
 0 7 9 6 2 7 2 5 7 3 5 8 

np.float64(0.837)